In [1]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import swisseph as swe


class ZodiacTransits:
    ZODIAC_SIGNS = ["Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo", 
                    "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"]
    ZODIAC_BOUNDS = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360]
    PLANETS = ['sun', 'moon', 'mercury', 'venus', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune']
    PLANET_IDS = {'sun': swe.SUN, 'moon': swe.MOON, 'mercury': swe.MERCURY,
                  'venus': swe.VENUS, 'mars': swe.MARS, 'jupiter': swe.JUPITER,
                  'saturn': swe.SATURN, 'uranus': swe.URANUS, 'neptune': swe.NEPTUNE}

    @staticmethod
    def get_zodiac_sign(longitude: float) -> str:
        """Determines the zodiac sign based on celestial longitude."""
        index = next(i for i, bound in enumerate(ZodiacTransits.ZODIAC_BOUNDS) if longitude < bound) - 1
        return ZodiacTransits.ZODIAC_SIGNS[index]



    def find_transits(self, start_date: datetime, end_date: datetime) -> list:
        """Calculates planetary transits within a given date range."""
        transits = []
        prev_signs = {planet: None for planet in self.PLANETS}
        prev_transit_start_date = {planet: start_date for planet in self.PLANETS}
        current_date = start_date

        while current_date <= end_date:
            self._process_day(
                current_date, prev_signs, prev_transit_start_date, transits
            )
            current_date += timedelta(days=1)

        self._finalize_transits(end_date, prev_signs, prev_transit_start_date, transits)
        return transits

    def _process_day(
        self,
        current_date: datetime,
        prev_signs: dict,
        prev_transit_start_date: dict,
        transits: list,
    ) -> None:
        """Processes transits for each planet on a given day."""
        processed_hours = []
        for hour in range(24):
            jul_day_ut = swe.julday(
                current_date.year, current_date.month, current_date.day, hour
            )
            processed_hours.append((jul_day_ut, hour))

        for jul_day_ut, hour in processed_hours:
            for planet in self.PLANETS:
                self._process_planet(
                    planet,
                    jul_day_ut,
                    current_date,
                    hour,
                    prev_signs,
                    prev_transit_start_date,
                    transits,
                )

    def _process_planet(
        self,
        planet: str,
        jul_day_ut: float,
        current_date: datetime,
        hour: int,
        prev_signs: dict,
        prev_transit_start_date: dict,
        transits: list,
    ) -> None:
        """Evaluates transit changes for a specific planet."""
        planet_id = self.PLANET_IDS[planet]
        ret_val, _ = swe.calc_ut(jul_day_ut, planet_id, swe.FLG_SWIEPH)
        lon = (ret_val[0] - 30) % 360
        sign = self.get_zodiac_sign(lon)
        if prev_signs[planet] != sign:
            if (
                prev_signs[planet] is not None
            ):  # Ensure there was a previous sign before adding the transit
                transits.append(
                    {
                        "planet": planet,
                        "sign": sign,
                        "start": prev_transit_start_date[planet],
                        "end": current_date.replace(hour=hour, minute=0, second=0),
                    }
                )
            prev_transit_start_date[planet] = current_date.replace(
                hour=hour, minute=0, second=0
            )
            prev_signs[planet] = sign

    def _finalize_transits(
        self,
        end_date: datetime,
        prev_signs: dict,
        prev_transit_start_date: dict,
        transits: list,
    ) -> None:
        """Finalizes the transit list by adding any ongoing transits at the end of the analysis period."""
        for planet, sign in prev_signs.items():
            if sign is not None:  # If the planet was in a sign at the end of the period
                transits.append(
                    {
                        "planet": planet,
                        "sign": sign,
                        "start": prev_transit_start_date[planet],
                        "end": end_date,
                    }
                )

    @classmethod
    def get_astrological_birth_details(cls, birth_datetime: datetime, latitude: float, longitude: float, timezone: str) -> dict:
        utc_datetime = birth_datetime.astimezone(ZoneInfo("UTC")) # Use ZoneInfo instead of pytz
        julian_day_ut = swe.julday(utc_datetime.year, utc_datetime.month, utc_datetime.day, utc_datetime.hour + utc_datetime.minute/60 + utc_datetime.second/3600)
        
        birth_details = {
            "object_reports": [],
            "date_time_and_location": {
                "date": birth_datetime.strftime("%Y-%m-%d"),
                "time": birth_datetime.strftime("%H:%M:%S"),
                "latitude": latitude,
                "longitude": longitude,
                "timezone": timezone
            }
        }

        for planet in cls.PLANETS:
            planet_id = cls.PLANET_IDS[planet]
            ret_val, _ = swe.calc_ut(julian_day_ut, planet_id, swe.FLG_SWIEPH)
            lon = ret_val[0] % 360
            sign = cls.get_zodiac_sign(lon)
            degree_within_sign = lon % 30
            whole_degrees = int(degree_within_sign)
            fractional_degrees = degree_within_sign - whole_degrees
            minutes = int(fractional_degrees * 60)
            seconds = int((fractional_degrees * 60 - minutes) * 60)
            is_in_retrograde = ret_val[3] < 0

            birth_details["object_reports"].append({
                "degrees": whole_degrees,
                "minutes": minutes,
                "seconds": seconds,
                "sign": sign,
                "object_type": planet.capitalize(),
                "is_in_retrograde": is_in_retrograde
            })

        return birth_details

In [2]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

def pretty_print_transits(transits, timezone=ZoneInfo("America/Los_Angeles")):
    """Prints the transits in a human-readable format, adjusted for the specified timezone."""
    for transit in transits[:20]:  # Limiting to the first 20 transits for brevity
        start_tz = transit["start"].astimezone(timezone)
        end_tz = transit["end"].astimezone(timezone)

        print(
            f"{transit['planet'].capitalize()} is in {transit['sign']} {start_tz:%B %d, %Y} - {end_tz:%B %d, %Y}"
        )

def filter_transits_for_today(transits, date, timezone=ZoneInfo("UTC")):
    """Filters transits that are active on the specified date, considering the timezone."""
    date_tz = date.astimezone(timezone)

    return [
        transit
        for transit in transits
        if transit["start"] <= date_tz < (transit["end"] + timedelta(days=1))
    ]

# Main execution
if __name__ == "__main__":
    # Instantiate the ZodiacTransits class and find transits
    zodiac_transits = ZodiacTransits()

    # Get astrological birth details
    BIRTH_DATETIME = datetime(1986, 4, 23, 0, 30, tzinfo=ZoneInfo("America/New_York"))
    BIRTH_LATITUDE = 40.8428759
    BIRTH_LONGITUDE = -73.2928943

    birth_details = ZodiacTransits.get_astrological_birth_details(BIRTH_DATETIME, BIRTH_LATITUDE, BIRTH_LONGITUDE, "America/New_York")
    for obj in birth_details['object_reports']:
        print(f"{obj['object_type']} {obj['sign']} {obj['degrees']}°{obj['minutes']}'{obj['seconds']}\"")
    print("\n")

    transits = zodiac_transits.find_transits(
        datetime(2020, 1, 1, tzinfo=ZoneInfo("UTC")), datetime(2026, 1, 1, tzinfo=ZoneInfo("UTC"))
    )
    today = datetime(2024, 2, 18, 0, 0, tzinfo=ZoneInfo("UTC"))
    transits_today = filter_transits_for_today(transits, today)
    pretty_print_transits(transits_today)

 


print("""
Actual: 
Moon is in Gemini February 16, 2024 - February 18, 2024
Sun is in Aquarius January 20, 2024 - February 18, 2024
Mercury is in Aquarius February 4, 2024 - February 22, 2024
Venus is in Aquarius February 16, 2024 - March 11, 2024
Mars is in Aquarius February 12, 2024 - March 22, 2024
Jupiter is in Taurus May 16, 2023 - May 25, 2024
Neptune is in Pisces Feb 3, 2022 - March 30, 2025
Saturn is in Pisces Mar 7, 2023 - May 24, 2025
Uranus is in Taurus Mar 6, 2019 - July 6, 2025
""")

swisseph


NameError: name 'pytz' is not defined